In [3]:
import requests

In [23]:
host_url = 'http://www.edupedia.pl'

In [24]:
root_url = host_url + '/map/dictionary/id/8_slownik_wyrazow_obcych.html'

In [5]:
root_html = requests.get(root_url)

In [11]:
from bs4 import BeautifulSoup
root_soup = BeautifulSoup(root_html.text, 'html.parser')

In [14]:
links_to_word_pages = root_soup.find_all('a', class_='lkyDescLink')

In [20]:
paths_to_words_pages = list(map(lambda a: a.attrs['href'], links_to_word_pages))

In [47]:
from bs4 import BeautifulSoup

def get_soup(url):
    text = requests.get(url).text
    return BeautifulSoup(text, 'html.parser')

In [60]:
def extract_definition(html):
    word_container = html.find(class_='wordsTitle')
    word = word_container.a.text.strip()
    definition = html.find(class_='wordsPtext').text.strip()

    return word, definition

In [62]:
def scrape_definition(url):
    soup = get_soup(url)
    return extract_definition(soup)

In [66]:
definitions = []

In [67]:
from tqdm.notebook import tqdm

for path in tqdm(paths_to_words_pages):
    url = host_url + path
    definition = scrape_definition(url)
    definitions.append((path, *definition))


  0%|          | 0/10582 [00:00<?, ?it/s]

In [74]:
import pickle

with open('definitions.pkl', 'wb') as f:
    pickle.dump(definitions, f)

In [85]:
import random
random.shuffle(definitions)

In [96]:
rows_to_write = tuple((d[1], d[2]) for d in definitions)

In [2]:
import csv
with open('thesaurus.csv', 'w') as f:
    writer = csv.writer(f,quoting=csv.QUOTE_ALL)
    writer.writerows(rows_to_write)